In [4]:
import pandas as pd
import numpy as np
import re

In [9]:
# Определяем рабочую директорию
work_dir = r"C:\Users\tatarikov\Desktop\R scripts\Daily report"
file_path = f"{work_dir}\\New queries.xlsx"
out_path = f"{work_dir}\\Processed_queries.xlsx"

# 1. Читаем Excel файл
df = pd.read_excel(file_path)

# 2. Фильтруем строки по пользователям
valid_users = [
    "Klim Tatarikov (tatarikov)"
]
df = df[df["User"].isin(valid_users)]

# 3. Исключаем строки, где "Day/Time" начинается с "31.03.2025"
df = df[~df["Day/Time"].astype(str).str.startswith("31.03.2025")]

# 4. Убираем окончание "(Repeat key x)" из поля "CRF"
df["CRF"] = df["CRF"].astype(str).apply(lambda x: re.sub(r"\(Repeat key \d+\)", "", x).strip())

# 5. Группировка по "Screening ID" и "Query number", подсчет "Description" в "Value type"
grouped = df.groupby(["Screening ID", "Query number"]).apply(
    lambda g: pd.Series({"Description_Count": (g["Value type"] == "Description").sum()})
).reset_index()

# 6. Фильтруем строки, где Description_Count = 1
filtered = grouped[grouped["Description_Count"] == 1]

# 7. Группируем по первым трём символам "Screening ID" и считаем количество строк
screening_grouped = filtered["Screening ID"].astype(str).str[:3].value_counts().reset_index()
screening_grouped.columns = ["Screening Prefix", "Count"]

# 8. Сохраняем результат в один файл с двумя листами
with pd.ExcelWriter(out_path, engine="xlsxwriter") as writer:
    filtered.to_excel(writer, sheet_name="Filtered Data", index=False)
    screening_grouped.to_excel(writer, sheet_name="Screening Grouped", index=False)

print(f"Обработка завершена. Данные сохранены в '{out_path}' с двумя листами.")


Обработка завершена. Данные сохранены в 'C:\Users\tatarikov\Desktop\R scripts\Daily report\Processed_queries.xlsx' с двумя листами.


C:\Users\tatarikov\AppData\Local\Temp\ipykernel_10236\2208950639.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = df.groupby(["Screening ID", "Query number"]).apply(
